In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

def read_the_data(directory_path):
    data = open(directory_path)
    return data

def convert_data__row_values_to_float(data):
    data_as_list = []
    shape_of_data = np.shape(data)[0]
    for row in range(0, shape_of_data):
        oneRow = data[0][row]
        oneRowList = oneRow.split()
        data_as_list.append(list(map(float, oneRowList)))
    
    return np.array(data_as_list)


 
def DBSCAN(set_of_points, Eps, min_points):
    
    
    no_of_points = np.shape(set_of_points)[0]
    labels = np.zeros(no_of_points, dtype = np.int8)
    cluster_id = 0
    
    '''initially, all the points are unclassified, i.e. have a label -1'''
    for i in range(0, no_of_points):
        if labels[i] == 0: 
            '''UNCLASSIFIED'''
            labels, bool_value = Expland_cluster(set_of_points, i, cluster_id, labels, Eps, min_points)
            if bool_value:
                cluster_id = cluster_id + 1
            
    return labels
    
    
        
        
def region_Query(set_of_points, index, Eps):
    Eps_neighbourhood = []
    
    no_of_points_in_dataset = np.shape(set_of_points)[0]
    
    for i in range(0, no_of_points_in_dataset):
        
        if np.linalg.norm(set_of_points[index] - set_of_points[i]) < Eps:
            Eps_neighbourhood.append(i)
        
    return (Eps_neighbourhood)



def change_cluster_Id(index, cluster_id, labels):
    labels[index] = cluster_id
    return labels
    
def Expland_cluster(set_of_points, index, cluster_id, labels, Eps, min_points):
    
    i = 0
    Eps_neighbourhood_of_a_point = region_Query(set_of_points, index, Eps)
    no_of_points_in_Eps_neighbourhood = np.shape(Eps_neighbourhood_of_a_point)[0]
    
    
    if no_of_points_in_Eps_neighbourhood < min_points: 
        
        '''NO CORE POINT'''
        labels = change_cluster_Id(index, -1, labels)
        return labels, False
    
    
    else: 
        '''all points in Eps_neighbourhood are density reachable from point in set_of_points at index'''
        for k in range(0, no_of_points_in_Eps_neighbourhood):
            labels = change_cluster_Id(Eps_neighbourhood_of_a_point[k], cluster_id, labels)
            
        
        required_index = ([i for i in range(0, no_of_points_in_Eps_neighbourhood) if Eps_neighbourhood_of_a_point[i] == index])
        
        del Eps_neighbourhood_of_a_point[required_index[0]]
        no_of_points_in_Eps_neighbourhood = no_of_points_in_Eps_neighbourhood - 1
    
        while i < len(Eps_neighbourhood_of_a_point):

            index_of_the_point_in_Eps_neigbourhood = Eps_neighbourhood_of_a_point[i]
            Eps_neighbourhood_of_point_in_while = region_Query(set_of_points, index_of_the_point_in_Eps_neigbourhood, Eps)
            no_of_points_in_Eps_neighbourhood_while = np.shape(Eps_neighbourhood_of_point_in_while)[0]

            if no_of_points_in_Eps_neighbourhood_while >= min_points:
                
                Eps_neighbourhood_of_a_point = list(set(Eps_neighbourhood_of_a_point + Eps_neighbourhood_of_point_in_while))
                
                for j in range(0, no_of_points_in_Eps_neighbourhood_while):
                    if labels[Eps_neighbourhood_of_point_in_while[j]] == 0:
                        labels = change_cluster_Id(Eps_neighbourhood_of_point_in_while[j], cluster_id, labels)
            
#                 for i in range(0, no_of_points_in_Eps_neighbourhood_while):
#                     index_of_a_point_in_the_points_set = Eps_neighbourhood_of_point_in_while[i]
#                     if labels[index_of_a_point_in_the_points_set] == 0 or lables[index_of_a_point_in_the_points_set] == -1:
                        
#                         if labels[index_of_a_point_in_the_points_set] == 0:
#                             Eps_neighbourhood_of_a_point.append(index_of_a_point_in_the_points_set)

#                         labels = change_cluster_Id(index_of_a_point_in_the_points_set, cluster_id, labels)
            i = i + 1
        return labels, True
                    
        
        
        
        


        
        
def get_indices_of_points_groupbed_by_cluster_id(labels):
    cluster_ids_in_labels = set(labels)
    labels_df = pd.DataFrame(labels, columns = ['id'])
    labels_grouped_by_cluster_id = labels_df.groupby(['id'])
    list_of_indices_grouped_by_ids = []

    for id in cluster_ids_in_labels:
        list_of_indices_grouped_by_ids.append(labels_grouped_by_cluster_id.get_group(id).index.values)

    return list_of_indices_grouped_by_ids

def calculate_center_points_of_cluster(list_of_indices_grouped_by_ids, data_df):
    
    no_of_clusters = len(list_of_indices_grouped_by_ids)
    center_of_clusters = []
    for i in range(0, no_of_clusters):
        points = np.array([data_df[i] for i in list_of_indices_grouped_by_ids[0]])
        center_of_clusters.append(points.mean(0))
    
    return np.array(center_of_clusters)




def plot_clusters(center_points_for_clusters, data_df, labels, is_kmeans, is_dataset1):
    X, Y = get_points_as_X_and_Y(data_df)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    scatter = ax.scatter(x = X, y = Y,c = labels, s=2)
    
        
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.colorbar(scatter)
    
    if is_kmeans and is_dataset1:
        fig.suptitle('data_1_kmeans', fontsize=10)
    
    elif not is_kmeans and is_dataset1:
        fig.suptitle('data_1_DBSCAN', fontsize=10)
    
    elif not is_kmeans and not is_dataset1:
        fig.suptitle('data_2_DBSCAN', fontsize=10)
    
    else:
        fig.suptitle('data_2_kmeans', fontsize=10)
        
    fig.show()


def get_points_as_X_and_Y(data_df):
    X = [point[0] for point in data_df]
    Y = [point[1] for point in data_df]
    return X, Y
                        
            

In [ ]:

data = pd.DataFrame(read_the_data("/dataset1.txt"))
data_df = convert_data__row_values_to_float(data)

In [ ]:
##################### 

kmeans = KMeans(n_clusters = 2, random_state =42).fit(data_df)
kmeans_labels_ = kmeans.labels_
kmeans_cluster_centers = kmeans.cluster_centers_
plot_clusters(kmeans_cluster_centers, data_df, kmeans_labels_, True, True)

##################### 

labels = DBSCAN(data_df, 3, 40)
list_of_indices_grouped_by_ids = get_indices_of_points_groupbed_by_cluster_id(labels)
center_points_of_clusters = calculate_center_points_of_cluster(list_of_indices_grouped_by_ids, data_df)
plot_clusters(center_points_of_clusters, data_df, labels, False, True)





In [ ]:
data2 = pd.DataFrame(read_the_data("/dataset2.txt"))
data2_df = convert_data__row_values_to_float(data2)

In [ ]:
######################

kmeans = KMeans(n_clusters = 2, random_state =42).fit(data2_df)
kmeans_labels_2 = kmeans.labels_
kmeans_cluster_centers_2 = kmeans.cluster_centers_
plot_clusters(kmeans_cluster_centers_2, data2_df, kmeans_labels_2, True, False)

#####################

labels2 = DBSCAN(data2_df, 3, 10)
list_of_indices_grouped_by_ids_2 = get_indices_of_points_groupbed_by_cluster_id(labels2)
center_points_of_clusters_2 = calculate_center_points_of_cluster(list_of_indices_grouped_by_ids_2, data2_df)
plot_clusters(center_points_of_clusters_2, data2_df, labels2, False, False)